Create files with monthly time series of mooring data in the Central and Dotson trough

In [1]:
import numpy as np
from numpy import ma

import xarray as xr
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib.dates as mdates
import matplotlib as mpl

import sys
import gsw

In [2]:
#-------------------------------------------------------------------
# Directories
#-------------------------------------------------------------------
voldir = '/Volumes/SamT5/PhD/data/'
mdir = voldir + 'moorings/dragomir_phd/'

localdir = '/Users/ocd1n16/PhD_git/'

auxscriptdir = localdir + 'aux_func/'
sys.path.append(auxscriptdir)
import aux_func_trend as fc

In [3]:
# time end points
s1_start = '2011-01-01'
s1_end = '2015-12-31'

ct_start = '2011-01-01'
ct_end = '2015-12-31'

# time/seasonal means: crop to integer number of years
# make sure the time mean period is inside the range of the time series
s1_tmean_start, s1_tmean_end = '2011-01-01', '2015-12-01'
ct_tmean_start, ct_tmean_end = '2011-01-01', '2015-12-01'

### DT

In [13]:
# S1 mooring data file
# 8 m grid
mfile = 's1_raw_uv_hourly_adcp_404m_540m_2010_2016.nc'

# depth range
dep_min_max = [412, 528]

with xr.open_dataset(mdir + mfile) as s1moor:
    print(s1moor.keys())

s1 = s1moor.sel(depth=slice(dep_min_max[0], dep_min_max[1]))

# average data daily
s1d = s1.resample(time="1D").mean()

# projection on rotated reference frame
tim, dep = s1d.u.shape
rot_u, rot_v = [np.ones((tim, dep)) for _ in range(2)]

for i in range(dep):
    rot_vel = fc.rotate_frame(s1d.u.isel(depth=i), s1d.v.isel(depth=i), 51, 'clockwise')
    rot_u[:, i] = rot_vel.u.values
    rot_v[:, i] = rot_vel.v.values
    
# add new components to existing dataset
s1d["u_rot"] = (("time", 'depth'), rot_u)
s1d["v_rot"] = (("time", "depth"), rot_v)

# average with depth + change units to cm/s to match CT
ts1d = s1d.mean("depth") * 1e2

KeysView(<xarray.Dataset>
Dimensions:  (time: 51839, depth: 18)
Coordinates:
  * time     (time) datetime64[ns] 2010-02-15T15:00:00 ... 2016-01-17T03:00:00
  * depth    (depth) float64 404.0 412.0 420.0 428.0 ... 516.0 524.0 532.0 540.0
Data variables:
    u        (time, depth) float64 ...
    v        (time, depth) float64 ...
Attributes:
    lon:      -116.358
    lat:      -72.468)


In [14]:
# monthly averages
mts1 = ts1d.resample(time="1MS").mean()

# crop anomalies to a new period
mts1 = mts1.sel(time=slice(s1_start, s1_end))

# time and seasonal mean
mts1_mean_crop = mts1.sel(time=slice(s1_tmean_start, s1_tmean_end))
mts1_timemean = mts1_mean_crop.mean("time")
mts1_meanseas = mts1_mean_crop.groupby("time.month").mean()
mts1_std = mts1_mean_crop.groupby("time.month").std(ddof=1)

# monthly time mean anomalies
mts1_anom = mts1 - mts1_timemean
mts1_mclim = mts1.groupby("time.month") - mts1_meanseas

#remove linear trend
us1_trend, us1_det = fc.trend_ci(mts1_anom.u_rot, 0.95)
_, us1_mclim_det = fc.trend_ci(mts1_mclim.u_rot, 0.95)

print("Variable: Monthly depth averaged velocity anomaly wrt the time mean")

print("\nTime mean: %s cm/s" % str(mts1_timemean.u.values))
print("Time mean computed between: " + s1_tmean_start + " and " + s1_tmean_end)

print("\nLength of time series: " + s1_start + " to " + s1_end)

print("\nLinear trend: %s mm/s per year" %(us1_trend.slope.values*10*365))
print("CI: %s mm/s per year " %(us1_trend.ci.values*10*365))
print("p-val: %s \n" %(us1_trend.p_val.values))

# standardise time series
us1_det_stand = us1_det / us1_det.std(ddof=1)
us1_mclim_det_stand = us1_mclim_det / us1_mclim_det.std(ddof=1)

Variable: Monthly depth averaged velocity anomaly wrt the time mean

Time mean: 0.9486501519145146 cm/s
Time mean computed between: 2011-01-01 and 2015-12-01

Length of time series: 2011-01-01 to 2015-12-31

Linear trend: -3.4989860325502358 mm/s per year
CI: 3.060750699862925 mm/s per year 
p-val: 0.025832295216441113 



### CT

In [4]:
#------------------------------------------------------------------
#            ~ ~ ~    OTHER MOORINGS     ~ ~ ~   
#------------------------------------------------------------------
# hourly
#bsr12_0 = xr.open_dataset(mdir_pd + 'bsr12_0_raw.nc') # 400 m
bsr12_1 = xr.open_dataset(mdir + 'bsr12_1_raw.nc') # 553 m
bsr13 = xr.open_dataset(mdir + 'bsr13a_raw.nc') # 338 m

istar1 = xr.open_dataset(mdir + 'istar1_raw.nc') # 581 m

troughW0 = xr.open_dataset(mdir + 'troughW0_raw.nc') # 423 m
troughW1 = xr.open_dataset(mdir + 'troughW1_raw.nc') # 555 m

troughE0 = xr.open_dataset(mdir + 'troughE0_raw.nc') # 467 m
troughE1 = xr.open_dataset(mdir + 'troughE1_raw.nc') # 595 m

# daily means
bsr12_d = bsr12_1.resample(time='1D').mean()
bsr13_d = bsr13.resample(time='1D').mean()
istar1_d = istar1.resample(time='1D').mean()
troughW1_d = troughW1.resample(time='1D').mean()
troughE1_d = troughE1.resample(time='1D').mean()

# vel components projected onto rotated frame; daily averages
bsr12_rot = fc.rotate_frame(bsr12_d.u, bsr12_d.v, 56, 'clockwise')
bsr13_rot = fc.rotate_frame(bsr13_d.u, bsr13_d.v, 17, 'anticlockwise')
istar1_rot = fc.rotate_frame(istar1_d.u, istar1_d.v, 47, 'clockwise')
troughW1_rot = fc.rotate_frame(troughW1_d.u, troughW1_d.v, 52, 'clockwise')
#troughE1_rot = fc.rotate_frame(troughE1_d.u, troughE1_d.v, 26, 'clockwise')

In [ ]:
# Distance in metres between the moorings in the central trough
dist1 = gsw.distance([istar1.lon.values, troughW1.lon.values], 
             [istar1.lat.values, troughW0.lat.values])

dist2 = gsw.distance([istar1.lon.values, bsr12_1.lon.values], 
             [istar1.lat.values, bsr12_1.lat.values])

dist3 = gsw.distance([istar1.lon.values, bsr13.lon.values], 
             [istar1.lat.values, bsr13.lat.values])

print("Distance between istar1 and trough W: %s m" % dist1)
print("Distance between istar1 and bsr12: %s m" % dist2)
print("Distance between istar1 and bsr13a: %s m" % dist3)

### Merge the moorings in CT after removing the time mean from every mooring

In [16]:
def get_anom(moor):
    anom = moor - moor.mean("time")
    return anom

In [17]:
bsr12_anom = get_anom(bsr12_rot.u)
bsr13_anom = get_anom(bsr13_rot.u)
istar1_anom = get_anom(istar1_rot.u)
tw1_anom = get_anom(troughW1_rot.u)

# concatenate moorings in the PITW/central trough
# subtract the time mean of every moooring before merging them 
ctd0 = xr.concat([bsr12_anom,
                  bsr13_anom,
                  istar1_anom,
                  tw1_anom], dim='time')

# resample daily and then monthly
ctd = ctd0.resample(time='1D').mean()

### File with daily data of merged CT product

In [20]:
ctd.attrs["units"] = "cm/s"
ctd.attrs['description'] = "Daily time means of velocity components at CT."
ctd.to_netcdf(mdir + "daily_ct_uv.nc")

#### monthly product

In [18]:
# monthly resampling
ctm = ctd.resample(time='1MS').mean()

In [21]:
ctm.to_netcdf(mdir + "monthly_ct_uv.nc")

In [19]:
# crop to the length of S1
ctm = ctm.sel(time=slice(s1_start, s1_end))

#--- ----- ----- check if there is any time mean
# crop time series to integer number of years
ctd_timemean_crop = ctd.sel(time=slice(ct_tmean_start, ct_tmean_end))
ctm_timemean_crop = ctm.sel(time=slice(ct_tmean_start, ct_tmean_end))

# time mean (time mean of individual moorings is already removed)
ctd_timemean = ctd_timemean_crop.mean("time")
ctm_timemean = ctm_timemean_crop.mean("time")

# mean seasonal cycle
ctd_meanseas = ctd_timemean_crop.groupby("time.month").mean()
ctm_meanseas = ctm_timemean_crop.groupby("time.month").mean()
ctm_std = ctm_timemean_crop.groupby("time.month").std(ddof=1)

print("Time mean of anomalies computed between: " + ct_tmean_start + " and " + ct_tmean_end)
print("daily average: %.7f cm/s" % ctd_timemean)
print("monthly average: %.7f cm/s" % ctm_timemean)

# monthly time mean anomalies
ctm_anom = ctm - ctm_timemean
ctm_mclim = ctm.groupby("time.month") - ctm_meanseas

#remove linear trend
ctm_trend, ctm_det = fc.trend_ci(ctm_anom, 0.95)
_, ctm_mclim_det = fc.trend_ci(ctm_mclim, 0.95)

print("\nLength of time series: " + ct_start + " to " + ct_end)

print("\nLinear trend: %s mm/s per year" %(ctm_trend.slope.values*10*365.25))
print("CI: %s mm/s per year " %(ctm_trend.ci.values*10*365.25))
print("p-val: %s \n" %(ctm_trend.p_val.values))

# standardise time series
ctm_det_stand = ctm_det / ctm_det.std(ddof=1)
ctm_mclim_det_stand = ctm_mclim_det / ctm_mclim_det.std(ddof=1)

Time mean of anomalies computed between: 2011-01-01 and 2015-12-01
daily average: -0.1516713 cm/s
monthly average: -0.0854489 cm/s

Length of time series: 2011-01-01 to 2015-12-31

Linear trend: 0.8847598986334414 mm/s per year
CI: 4.780883754399861 mm/s per year 
p-val: 0.7125017743773789 



In [20]:
au = xr.Dataset({"dt" : ("time", mts1.u_rot.values), 
               "dt_anom" : ("time", mts1_anom.u_rot.values),
               "dt_anom_det" : ("time", us1_det.values),
                "dt_anom_det_stand" : ("time", us1_det_stand.values),
               "ct" : ("time", ctm.values),
               "ct_anom" : ("time", ctm_anom.values),
               "ct_anom_det" : ("time", ctm_det.values),
                "ct_anom_det_stand" : ("time", ctm_det_stand.values)},
               coords = {"time" : mts1.time.values})
au.dt["long_name"] = "Dotson Trough"
au.ct["long_name"] = "Central Trough"
au.attrs["units"] = "cm/s"

au.to_netcdf(mdir + "amundsen_undercurrent_monthly_2011jan_2015dec.nc")